In [1]:
from google.colab import files
uploaded = files.upload()

Saving concrete_data.csv to concrete_data.csv


In [2]:
import pandas as pd
import numpy as np


In [3]:
data = pd.read_csv(r'concrete_data.csv')
data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


## Feature Selection

In [20]:
# Read Carefully
x = data.iloc[:, :8].values
y = data.iloc[:, 8].values.reshape(-1, 1)


In [21]:
print(x)

[[ 540.     0.     0.  ... 1040.   676.    28. ]
 [ 540.     0.     0.  ... 1055.   676.    28. ]
 [ 332.5  142.5    0.  ...  932.   594.   270. ]
 ...
 [ 148.5  139.4  108.6 ...  892.4  780.    28. ]
 [ 159.1  186.7    0.  ...  989.6  788.9   28. ]
 [ 260.9  100.5   78.3 ...  864.5  761.5   28. ]]


In [22]:
print(y)


[[79.99]
 [61.89]
 [40.27]
 ...
 [23.7 ]
 [32.77]
 [32.4 ]]


In [23]:
print(np.shape(x))
print(np.shape(y))


(1030, 8)
(1030, 1)


In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2000)


#### We are taking XGBoost algorithm as our problem is regression problem.

In [27]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(random_state=2000)


#### To define the parameters require for hyperparameter tuning

#### In the grid search process, we are going to pick each value for the number of trees to train a model and we are going to find which value works the best. Given below are the 4 hyperparameters used in XGBoost, and we are going to train 72 models in our grid search CV as 3x3x2x4 = 72


In [29]:
# In the grid search process, we are going to pick each value for the number of trees to train a model and we are going to find which value works the best.
# given below are the 4 hyperparameters used in XGBoost, and we are going to train 72 models in our grid search CV as 3x3x2x4 = 72

search_space = {
    "n_estimators " : [100, 200, 500], #3
    "max_depth" : [3, 6, 9], #3
    "gamma" : [0.01, 0.1], #3
    "learning_rate": [0.001, 0.01, 0.1, 1] #

}

In [31]:
from sklearn.model_selection import GridSearchCV

GS = GridSearchCV(estimator = xgb_model,
                  param_grid= search_space,
                  scoring = ["r2", "neg_root_mean_squared_error"], #sklearn.metrics.Scorers.keys()
                  refit="r2",
                  cv = 5,
                  verbose = 4)
                  

In [32]:
GS.fit(x_train, y_train)


Fitting 5 folds for each of 72 candidates, totalling 360 fits
[04:36:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV 1/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators =100; neg_root_mean_squared_error: (test=-36.290) r2: (test=-3.291) total time=   0.1s
[04:36:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV 2/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators =100; neg_root_mean_squared_error: (test=-36.004) r2: (test=-3.082) total time=   0.0s
[04:36:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV 3/5] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators =100; neg_root_mean_squared_error: (test=-35.105) r2: (test=-3.692) total time=   0.0s
[04:36:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear i

GridSearchCV(cv=5, estimator=XGBRegressor(random_state=2000),
             param_grid={'gamma': [0.01, 0.1],
                         'learning_rate': [0.001, 0.01, 0.1, 1],
                         'max_depth': [3, 6, 9],
                         'n_estimators ': [100, 200, 500]},
             refit='r2', scoring=['r2', 'neg_root_mean_squared_error'],
             verbose=4)

#### The above recurring model shows details of the hyperparameters

#### Now we are determining the best hyperparameter to tune the model

In [33]:
print(GS.best_estimator_)

XGBRegressor(gamma=0.01, max_depth=6, n_estimators =100, random_state=2000)


In [34]:
print(GS.best_params_)

{'gamma': 0.01, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators ': 100}


#### To get best score, we are determining the rms value there

#### Number of parameters i.e param_n is the number of trees used in XGBoost

In [35]:
print(GS.best_score_)


0.9244376292464519


In [36]:
df = pd.DataFrame(GS.cv_results_)
df = df.sort_values('rank_test_r2')
df.to_csv("cv_results.csv")
